In [ ]:
import pandas as pd
from huggingface_hub import InferenceClient
import time
import os
from pathlib import Path

# ================================
# CONFIGURATION
# ================================
FILE_INDEX = ""
CSV_FILE_PATH = "" + FILE_INDEX + ".csv"
HF_TOKEN = ""
MODEL_NAME = ""
ENDPOINT_URL = ""
OUTPUT_DIR = "/content"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ================================
# PROMPT GENERATION FUNCTION
# ================================
def create_prompts(sentence1, sentence2, task, studied_quantity, person_a="PERSON_A", person_b="PERSON_B"):
    sentence1 = sentence1.replace("[PERSON_A]", person_a).replace("[PERSON_B]", person_b)
    sentence2 = sentence2.replace("[PERSON_A]", person_a).replace("[PERSON_B]", person_b)
    combined_sentences = f"{sentence1} {sentence2}"

    templates = {
        "simple": "How does the amount of {studied_quantity} {person_b} spends on {task} compare to that of {person_a}?",
        "more_control": "Does {person_b} spend more {studied_quantity} on {task} than {person_a}?",
        "less_control": "Does {person_b} spend less {studied_quantity} on {task} than {person_a}?",
        "equal_control": "Does {person_b} spend equal {studied_quantity} on {task} as {person_a}?",
        "more_test": "{person_b} spends more {studied_quantity} on {task} than {person_a} in several instances. Does {person_b} spend more {studied_quantity} on {task} than {person_a}?",
        "equal_test": "{person_a} and {person_b} spend different amounts of {studied_quantity} on {task}, but do they spend the same total {studied_quantity} on {task}?",
        "less_test": "{person_b} spends less {studied_quantity} on {task} than {person_a} in several instances. Does {person_b} spend less {studied_quantity} on {task} than {person_a}?"
    }

    answer_instructions = "\nA) less\nB) more\nC) equal\nONLY RETURN ONE WORD ANSWER THAT IS THE TRUE OPTION'S TEXT [less, more, or equal], WITHOUT ANY EXPLANATION."
    # answer_instructions = (
    #     "\nA) less\nB) more\nC) equal\n\n"
    #     "Return only a JSON object with a single field `answer`, whose value is one of: less, more, or equal. "
    #     "Example: {\"answer\": \"less\"}. "
    #     "DO NOT include any explanation, preamble, or formatting—only the JSON object."
    # )

    # answer_instructions = (
    # "\nA) less\nB) more\nC) equal\n\n"
    # "Return only a JSON object with two fields:\n"
    # " - `reasoning`: a concise explanation of how you reached your conclusion.\n"
    # " - `answer`: one of the following strings: less, more, or equal.\n"
    # "Example:\n"
    # "{\n  \"reasoning\": \"Person B spent more time lifting weights based on both sentences.\",\n  \"answer\": \"more\"\n}\n"
    # "Do not include any additional text outside of the JSON object."
    # )


    # answer_instructions = (
    # "\nA) less\nB) more\nC) equal\n\n"
    # "let's think step by step."
    # )

    prompts_beginning = {
        f"{key}_beginning": f"{template.format(person_a=person_a, person_b=person_b, studied_quantity=studied_quantity, task=task)}\n\n{combined_sentences}{answer_instructions}"
        for key, template in templates.items()
    }

    prompts_end = {
        f"{key}_end": f"{combined_sentences}\n\n{template.format(person_a=person_a, person_b=person_b, studied_quantity=studied_quantity, task=task)}{answer_instructions}"
        for key, template in templates.items()
    }

    return {**prompts_beginning, **prompts_end}
# ================================
# MAIN PROCESSING FUNCTION
# ================================
def process_claude_prompts_hf():
    df = pd.read_csv(CSV_FILE_PATH)
    results = []

    # Load checkpoint if exists
    checkpoint_file = os.path.join(OUTPUT_DIR, f"{MODEL_NAME.replace('/', '_')}_{FILE_INDEX}_CHECKPOINT_reason.csv")
    if Path(checkpoint_file).exists():
        results_df = pd.read_csv(checkpoint_file)
        processed_ids = set(results_df['request_id'].tolist())
        results = results_df.to_dict("records")
        print(f"Loaded checkpoint with {len(processed_ids)} processed prompts")
    else:
        processed_ids = set()

    # Initialize HF client
    client = InferenceClient(ENDPOINT_URL, token=HF_TOKEN)
    print(f"Using endpoint: {ENDPOINT_URL}")

    for index, row in df.iterrows():
        row_id = row['ID']
        sentence1 = row['sentence1']
        sentence2 = row['sentence2']
        task = row.get('task', "lifting weights")
        studied_quantity = row.get('studied_quantity', "energy")

        prompts = create_prompts(sentence1, sentence2, task, studied_quantity)

        for prompt_type, prompt in prompts.items():
            request_id = f"{row_id}_{prompt_type}"
            if request_id in processed_ids:
                continue  # Skip if already done

            print(f"Processing: {request_id}")

            response = None
            for attempt in range(3):
                try:
                    response = client.text_generation(
                        prompt=prompt,
                        max_new_tokens=600,
                        temperature=0.1,
                        # top_p=0.9,
                    )
                    break
                except Exception as e:
                    print(f"Error on {request_id} (Attempt {attempt+1}/3): {e}")
                    time.sleep(10 * (attempt + 1))
            if response is None:
                response = "ERROR"

            results.append({
                "request_id": request_id,
                "row_id": row_id,
                "prompt_type": prompt_type,
                "prompt": prompt,
                "response": response.strip()
            })

            # Save after every prompt
            pd.DataFrame(results).to_csv(checkpoint_file, index=False)

            # Delay between requests
            time.sleep(1)

    final_df = pd.DataFrame(results)
    final_output_path = os.path.join(OUTPUT_DIR, f"{MODEL_NAME.replace('/', '_')}_{FILE_INDEX}_reason.csv")
    final_df.to_csv(final_output_path, index=False)
    print(f"\nFinal results saved to: {final_output_path}")
    return final_df

if __name__ == "__main__":
    process_claude_prompts_hf()

Using endpoint: https://zannnw6b0gh8o49c.us-east-1.aws.endpoints.huggingface.cloud
Processing: 1_simple_beginning
Processing: 1_more_control_beginning
Processing: 1_less_control_beginning
Processing: 1_equal_control_beginning
Processing: 1_more_test_beginning
Processing: 1_equal_test_beginning
Processing: 1_less_test_beginning
Processing: 1_simple_end
Processing: 1_more_control_end
Processing: 1_less_control_end
Processing: 1_equal_control_end
Processing: 1_more_test_end
Processing: 1_equal_test_end
Processing: 1_less_test_end
Processing: 2_simple_beginning
Processing: 2_more_control_beginning
Processing: 2_less_control_beginning
Processing: 2_equal_control_beginning
Processing: 2_more_test_beginning
Processing: 2_equal_test_beginning
Processing: 2_less_test_beginning
Processing: 2_simple_end
Processing: 2_more_control_end
Processing: 2_less_control_end
Processing: 2_equal_control_end
Processing: 2_more_test_end
Processing: 2_equal_test_end
Processing: 2_less_test_end
Processing: 3_sim